In [1]:
import sys
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().resolve().parents[0]
sys.path.insert(0, str(PROJECT_ROOT))

In [2]:
from src.cohort import load_aki_cohort
from src.utils import (
    add_icu_los_days,
    add_dialysis_flag,
    add_early_dopamine_flag,
    add_sofa_score,
    add_vasopressor_flags,
    add_mechanical_ventilation_flag,
    add_early_late_dialysis_flags,
    extract_dialysis_timing,              # falls in src.utils vorhanden
    add_dialysis_near_icu_discharge_flag,
    recode_ethnicity,
    add_rrt_persistence_near_discharge,
)

# 1) Load cohort
df_aki = load_aki_cohort()

# 2) Ensure time columns are datetime (super wichtig!)
for c in ["intime", "outtime", "admittime", "dischtime", "deathtime"]:
    if c in df_aki.columns:
        df_aki[c] = df_aki[c].astype("datetime64[ns]")

# 3) Add baseline / outcomes
df_aki = add_icu_los_days(df_aki)

# 4) Dialysis "ever" flag (pragmatic; ICD + MV)
df_aki = add_dialysis_flag(df_aki)

# 5) SOFA + interventions
df_aki = add_sofa_score(df_aki)
df_aki = add_mechanical_ventilation_flag(df_aki)

# 6) Vasopressors / dopamine early flags
df_aki = add_vasopressor_flags(df_aki, window_hours=24)
df_aki = add_early_dopamine_flag(df_aki, window_hours=24)

# 7) Ethnicity grouping
df_aki = recode_ethnicity(df_aki)   # sollte eine neue Spalte wie ethnicity_grp erzeugen

# 8) Timing-aware dialysis (nur MV Events)
df_aki = add_early_late_dialysis_flags(
    df_aki,
    window_hours=24,
    include_inputevents=True,
    allow_negative_hours=False
)

# 9) Optional: Dialysis start/end/duration (falls du die Funktion hast)
# df_aki = extract_dialysis_timing(df_aki)

# 10) Dialysis near ICU discharge (last 6h)
df_aki = add_dialysis_near_icu_discharge_flag(
    df_aki,
    hours_before_discharge=6,
    include_inputevents=True
)
df_aki = extract_dialysis_timing(df_aki)
df_aki = add_rrt_persistence_near_discharge(
    df_aki,
    hours_before_discharge=6
) 

# 11) Quick sanity checks
print("Rows:", len(df_aki))
print(df_aki[[
    "dialysis", "dialysis_timed", "dialysis_icd_only",
    "early_dialysis", "late_dialysis",
    "any_vasopressor", "mechanical_ventilation"
]].value_counts().head(15))

df_aki.head()


Rows: 10485
dialysis  dialysis_timed  dialysis_icd_only  early_dialysis  late_dialysis  any_vasopressor  mechanical_ventilation
0         0               0                  0               0              0                0                         4691
                                                                                             1                         3145
                                                                            1                1                          902
1         0               1                  0               0              0                1                          449
0         0               0                  0               0              1                0                          357
1         0               1                  0               0              0                0                          311
          1               0                  0               1              1                1                          137
    

,subject_id,hadm_id,icustay_id,intime,outtime,gender,dob,admittime,dischtime,deathtime,...,late_dialysis,dialysis_icd_only,dialysis_last_6h,dialysis_start,dialysis_end,dialysis_duration_hours,rrt_any_in_last6h,rrt_active_at_outtime,rrt_persistent_last6h,rrt_max_overlap_hours_in_window
0,3,145834,211552,2101-10-20 19:10:11,2101-10-26 20:43:09,M,2025-04-11,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,...,0,0,0,NaT,NaT,NaN,0,0,0,0.0
1,9,150750,220597,2149-11-09 13:07:02,2149-11-14 20:52:14,M,2108-01-26,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,...,0,0,0,NaT,NaT,NaN,0,0,0,0.0
2,21,109451,217847,2134-09-11 20:50:04,2134-09-17 18:28:32,M,2047-04-04,2134-09-11 12:17:00,2134-09-24 16:15:00,NaT,...,0,1,0,NaT,NaT,NaN,0,0,0,0.0
3,38,185910,248910,2166-08-10 00:29:36,2166-09-04 13:39:23,M,2090-08-31,2166-08-10 00:28:00,2166-09-04 11:30:00,NaT,...,0,0,0,NaT,NaT,NaN,0,0,0,0.0
4,52,190797,261857,2191-01-10 02:12:55,2191-01-11 22:37:31,M,2152-11-26,2191-01-10 02:12:00,2191-01-19 16:10:00,NaT,...,0,0,0,NaT,NaT,NaN,0,0,0,0.0


In [3]:
sub = df_aki[
    (df_aki["sofa"] >= 8) &
    (df_aki["sofa_renal"] >= 3) 
].copy()

print(sub["ethnicity_grp"].value_counts())
sub["sofa"].describe()


ethnicity_grp
White       1062
Other        226
Black        172
Hispanic      39
Asian         30
Name: count, dtype: int64


count    1529.000000
mean       11.302158
std         2.946335
min         8.000000
25%         9.000000
50%        11.000000
75%        13.000000
max        22.000000
Name: sofa, dtype: float64

In [4]:
out = (
    sub
    .groupby(["ethnicity_grp", "any_vasopressor"])
    ["rrt_persistent_last6h"]
    .mean()
    .reset_index()
)

out["rate_pct"] = out["rrt_persistent_last6h"] * 100
print(out)


  ethnicity_grp  any_vasopressor  rrt_persistent_last6h   rate_pct
0         Asian                0               0.176471  17.647059
1         Asian                1               0.230769  23.076923
2         Black                0               0.096774   9.677419
3         Black                1               0.229167  22.916667
4      Hispanic                0               0.178571  17.857143
5      Hispanic                1               0.454545  45.454545
6         Other                0               0.046053   4.605263
7         Other                1               0.391892  39.189189
8         White                0               0.084000   8.400000
9         White                1               0.282051  28.205128


In [5]:
sub.groupby(["ethnicity_grp", "any_vasopressor"])["rrt_persistent_last6h"].mean()
print(sub.groupby(["ethnicity_grp", "any_vasopressor"])["rrt_persistent_last6h"].value_counts())


ethnicity_grp  any_vasopressor  rrt_persistent_last6h
Asian          0                0                         14
                                1                          3
               1                0                         10
                                1                          3
Black          0                0                        112
                                1                         12
               1                0                         37
                                1                         11
Hispanic       0                0                         23
                                1                          5
               1                0                          6
                                1                          5
Other          0                0                        145
                                1                          7
               1                0                         45
                               

In [6]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy.stats import mannwhitneyu
from src.db import q

# ============================================================
# Flüssigkeitsmanagement: kurze, sinnvolle Outcomes/Exposures
# Benötigt: q(sql) aus src.db, und df_aki mit icustay_id, intime, outtime, age, sofa, mechanical_ventilation,
#          hospital_mortality, rrt_persistent_last6h (oder late_dialysis), icu_los_days
# ============================================================

def add_fluid_features_first24h(df_aki: pd.DataFrame, window_hours: float = 24.0) -> pd.DataFrame:
    """
    Adds simple fluid management features within the first 24h after ICU admission:
      - fluid_in_ml_24h (total IV fluid volume from inputevents_mv)
      - urine_out_ml_24h (urine volume from outputevents)
      - net_balance_ml_24h = in - out
      - fluid_type_has_hetarch_24h (proxy for HES/hetastarch exposure; label-based)
    Notes:
      - This is a pragmatic, label-based approach (good for exploratory/sensitivity analyses).
      - You may want to refine itemid lists later for strict reproducibility.
    """
    df = df_aki.copy()
    required = {"icustay_id", "intime"}
    missing = sorted(required - set(df.columns))
    if missing:
        raise ValueError(f"df_aki fehlt Spalten: {missing}")

    # --- Inputs: crystalloids/colloids etc. (broad; you can narrow later)
    inp = q("""
        SELECT ie.icustay_id, ie.starttime, ie.endtime, ie.amount, ie.amountuom, di.label
        FROM inputevents_mv ie
        JOIN d_items di ON ie.itemid = di.itemid
        WHERE ie.amount IS NOT NULL
          AND LOWER(ie.amountuom) IN ('ml','milliliter','milliliters')
    """)
    inp = inp.dropna(subset=["icustay_id", "starttime", "amount"]).copy()

    # --- Outputs: urine (outputevents is in MIMIC-III)
    out = q("""
        SELECT oe.icustay_id, oe.charttime, oe.value, oe.valueuom
        FROM outputevents oe
        WHERE oe.value IS NOT NULL
          AND LOWER(oe.valueuom) IN ('ml','milliliter','milliliters')
    """)
    out = out.dropna(subset=["icustay_id", "charttime", "value"]).copy()

    # Merge intime for windowing
    base = df[["icustay_id", "intime"]].dropna().copy()

    inp = inp.merge(base, on="icustay_id", how="inner")
    inp["h"] = (inp["starttime"] - inp["intime"]).dt.total_seconds() / 3600
    inp24 = inp[(inp["h"] >= 0) & (inp["h"] <= window_hours)].copy()

    out = out.merge(base, on="icustay_id", how="inner")
    out["h"] = (out["charttime"] - out["intime"]).dt.total_seconds() / 3600
    out24 = out[(out["h"] >= 0) & (out["h"] <= window_hours)].copy()

    # Total input volume
    fluid_in = inp24.groupby("icustay_id", as_index=False)["amount"].sum().rename(columns={"amount": "fluid_in_ml_24h"})

    # Urine output (note: outputevents contains many types; this is broad. If you want urine-only, filter by itemids/labels)
    urine_out = out24.groupby("icustay_id", as_index=False)["value"].sum().rename(columns={"value": "urine_out_ml_24h"})

    # HES/hetastarch proxy in first 24h
    inp24["label_l"] = inp24["label"].astype(str).str.lower()
    het_mask = inp24["label_l"].str.contains("hetastarch|hetarch|hydroxyethyl|hes", regex=True)
    het = (
        inp24.loc[het_mask]
        .groupby("icustay_id", as_index=False)
        .size()
        .rename(columns={"size": "hetastarch_events_24h"})
    )

    df = df.merge(fluid_in, on="icustay_id", how="left")
    df = df.merge(urine_out, on="icustay_id", how="left")
    df = df.merge(het, on="icustay_id", how="left")

    df["fluid_in_ml_24h"] = df["fluid_in_ml_24h"].fillna(0.0)
    df["urine_out_ml_24h"] = df["urine_out_ml_24h"].fillna(0.0)
    df["hetastarch_events_24h"] = df["hetastarch_events_24h"].fillna(0).astype(int)
    df["fluid_type_has_hetarch_24h"] = (df["hetastarch_events_24h"] > 0).astype(int)

    df["net_balance_ml_24h"] = df["fluid_in_ml_24h"] - df["urine_out_ml_24h"]

    return df


def analyze_fluid_management_outcomes(df: pd.DataFrame) -> None:
    """
    Runs a compact set of sensible analyses:
      A) Compare net balance (24h) in those with/without persistent RRT (last6h)
      B) Logistic regression: rrt_persistent_last6h ~ net_balance + age + sofa + MV
      C) Logistic regression: hospital_mortality ~ net_balance + age + sofa + MV
      D) Sensitivity: HES exposure (binary) vs outcomes
    """
    need = {"net_balance_ml_24h", "fluid_in_ml_24h", "rrt_persistent_last6h", "hospital_mortality",
            "age", "sofa", "mechanical_ventilation", "fluid_type_has_hetarch_24h"}
    missing = sorted(need - set(df.columns))
    if missing:
        raise ValueError(f"Fehlende Spalten für Analyse: {missing}")

    d = df.dropna(subset=["age", "sofa", "mechanical_ventilation", "rrt_persistent_last6h", "hospital_mortality"]).copy()

    # --- A) Nonparametric comparison
    x1 = d.loc[d["rrt_persistent_last6h"] == 1, "net_balance_ml_24h"]
    x0 = d.loc[d["rrt_persistent_last6h"] == 0, "net_balance_ml_24h"]
    stat, p = mannwhitneyu(x1, x0, alternative="two-sided")
    print(f"Net balance 24h (ml): RRT-persistent median={x1.median():.0f} vs non-persistent median={x0.median():.0f}, p={p:.3g}")

    # scale net balance for interpretability (per +1000 ml)
    d["net_balance_l_24h"] = d["net_balance_ml_24h"] / 1000.0
    d["fluid_in_l_24h"] = d["fluid_in_ml_24h"] / 1000.0

    # --- B) RRT persistence model
    m_rrt = smf.logit(
        "rrt_persistent_last6h ~ net_balance_l_24h + age + sofa + mechanical_ventilation",
        data=d
    ).fit(disp=False)
    print("\nModel: rrt_persistent_last6h ~ net_balance (per +1L) + age + sofa + MV")
    print(m_rrt.summary())

    # --- C) Mortality model
    m_mort = smf.logit(
        "hospital_mortality ~ net_balance_l_24h + age + sofa + mechanical_ventilation",
        data=d
    ).fit(disp=False)
    print("\nModel: hospital_mortality ~ net_balance (per +1L) + age + sofa + MV")
    print(m_mort.summary())

    # --- D) Sensitivity: HES/hetastarch exposure
    # crude rates
    e1 = d[d["fluid_type_has_hetarch_24h"] == 1]
    e0 = d[d["fluid_type_has_hetarch_24h"] == 0]
    if len(e1) >= 20 and len(e0) >= 20:
        print("\nHES/hetastarch exposure within 24h (crude):")
        print(f"  n_exposed={len(e1)}, n_unexposed={len(e0)}")
        print(f"  RRT persistent: {e1['rrt_persistent_last6h'].mean()*100:.1f}% vs {e0['rrt_persistent_last6h'].mean()*100:.1f}%")
        print(f"  Mortality:      {e1['hospital_mortality'].mean()*100:.1f}% vs {e0['hospital_mortality'].mean()*100:.1f}%")

        m_hes_rrt = smf.logit(
            "rrt_persistent_last6h ~ fluid_type_has_hetarch_24h + age + sofa + mechanical_ventilation",
            data=d
        ).fit(disp=False)
        print("\nModel: rrt_persistent_last6h ~ HES_exposure + age + sofa + MV")
        print(m_hes_rrt.summary())
    else:
        print("\nHES/hetastarch exposure: too few exposed/unexposed for stable modeling.")


# ===========================
# Usage
# ===========================
df_aki = add_fluid_features_first24h(df_aki, window_hours=24)
analyze_fluid_management_outcomes(df_aki)


Net balance 24h (ml): RRT-persistent median=5913 vs non-persistent median=-507, p=8.62e-104

Model: rrt_persistent_last6h ~ net_balance (per +1L) + age + sofa + MV
                             Logit Regression Results                            
Dep. Variable:     rrt_persistent_last6h   No. Observations:                10485
Model:                             Logit   Df Residuals:                    10480
Method:                              MLE   Df Model:                            4
Date:                   Mon, 26 Jan 2026   Pseudo R-squ.:                  0.1456
Time:                           21:12:52   Log-Likelihood:                -1602.5
converged:                          True   LL-Null:                       -1875.6
Covariance Type:               nonrobust   LLR p-value:                6.599e-117
                             coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------

In [7]:
df = df_aki.copy()

# Quartile der Netto-Bilanz
q25 = df["net_balance_ml_24h"].quantile(0.25)
q75 = df["net_balance_ml_24h"].quantile(0.75)

df["fluid_group"] = pd.NA
df.loc[df["net_balance_ml_24h"] <= q25, "fluid_group"] = "Low"
df.loc[df["net_balance_ml_24h"] >= q75, "fluid_group"] = "High"

# nur extreme Gruppen behalten
df_fg = df[df["fluid_group"].isin(["Low", "High"])].copy()

print(df_fg["fluid_group"].value_counts())
print(df_fg.groupby("fluid_group")["net_balance_ml_24h"].median())


fluid_group
Low     2623
High    2622
Name: count, dtype: int64
fluid_group
High    7491.96131
Low    -2920.00000
Name: net_balance_ml_24h, dtype: float64


In [8]:
from scipy.stats import fisher_exact

gH = df_fg[df_fg["fluid_group"] == "High"]
gL = df_fg[df_fg["fluid_group"] == "Low"]

rate_H = gH["rrt_persistent_last6h"].mean()
rate_L = gL["rrt_persistent_last6h"].mean()

a = gH["rrt_persistent_last6h"].sum()
b = len(gH) - a
c = gL["rrt_persistent_last6h"].sum()
d = len(gL) - c

_, p = fisher_exact([[a, b], [c, d]])

print(
    f"Persistent RRT: High fluid={rate_H*100:.1f}% vs "
    f"Low fluid={rate_L*100:.1f}% "
    f"(Δ {(rate_H-rate_L)*100:+.1f}pp), p={p:.3g}"
)


Persistent RRT: High fluid=11.1% vs Low fluid=0.5% (Δ +10.6pp), p=3.53e-72


In [9]:
rate_H = gH["hospital_mortality"].mean()
rate_L = gL["hospital_mortality"].mean()

a = gH["hospital_mortality"].sum()
b = len(gH) - a
c = gL["hospital_mortality"].sum()
d = len(gL) - c

_, p = fisher_exact([[a, b], [c, d]])

print(
    f"Hospital mortality: High fluid={rate_H*100:.1f}% vs "
    f"Low fluid={rate_L*100:.1f}% "
    f"(Δ {(rate_H-rate_L)*100:+.1f}pp), p={p:.3g}"
)


Hospital mortality: High fluid=23.8% vs Low fluid=16.3% (Δ +7.5pp), p=9.83e-12


In [10]:
# SOFA-Gruppen
df_fg["sofa_grp"] = pd.cut(
    df_fg["sofa"],
    bins=[-1, 4, 7, 12, 24],
    labels=["Mild (≤4)", "Moderate (5–7)", "Moderate (8–12)", "Severe (≥13)"]
)

for s, dsub in df_fg.groupby("sofa_grp"):
    if len(dsub) < 50:
        continue

    gH = dsub[dsub["fluid_group"] == "High"]
    gL = dsub[dsub["fluid_group"] == "Low"]

    if len(gH) < 20 or len(gL) < 20:
        continue

    print(f"\nSOFA group: {s}")
    print(
        f"  Persistent RRT: "
        f"High={gH['rrt_persistent_last6h'].mean()*100:.1f}% | "
        f"Low={gL['rrt_persistent_last6h'].mean()*100:.1f}%"
    )
    print(
        f"  Mortality: "
        f"High={gH['hospital_mortality'].mean()*100:.1f}% | "
        f"Low={gL['hospital_mortality'].mean()*100:.1f}%"
    )
    print("  N:", len(dsub))



SOFA group: Mild (≤4)
  Persistent RRT: High=2.4% | Low=0.2%
  Mortality: High=8.1% | Low=8.9%
  N: 1869

SOFA group: Moderate (5–7)
  Persistent RRT: High=5.8% | Low=1.1%
  Mortality: High=15.9% | Low=13.5%
  N: 1597

SOFA group: Moderate (8–12)
  Persistent RRT: High=17.3% | Low=0.3%
  Mortality: High=33.6% | Low=26.6%
  N: 1403

SOFA group: Severe (≥13)
  Persistent RRT: High=31.0% | Low=1.2%
  Mortality: High=58.8% | Low=67.1%
  N: 376


C:\Users\Simon\AppData\Local\Temp\ipykernel_49048\3907369229.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for s, dsub in df_fg.groupby("sofa_grp"):


In [12]:
sub = df_fg[
    (df_fg["sofa"] >= 8) &
    (df_fg["sofa"] <= 12) &
    (df_fg["fluid_group"].isin(["High", "Low"]))
].copy()

print(len(sub))
print(sub["fluid_group"].value_counts())


1403
fluid_group
High    791
Low     612
Name: count, dtype: int64


In [13]:
from scipy.stats import mannwhitneyu

gH = sub[sub["fluid_group"] == "High"]
gL = sub[sub["fluid_group"] == "Low"]

med_H = gH["sofa_renal"].mean ()
med_L = gL["sofa_renal"].mean()
print(med_H, med_L)

stat, p = mannwhitneyu(
    gH["sofa_renal"].dropna(),
    gL["sofa_renal"].dropna(),
    alternative="two-sided"
)

print(
    f"Renal SOFA (baseline): "
    f"High={med_H:.1f} vs Low={med_L:.1f}, p={p:.4f}"
)


2.2199747155499368 1.8774509803921569
Renal SOFA (baseline): High=2.2 vs Low=1.9, p=0.0000


In [14]:
pd.crosstab(
    sub["sofa_renal"],
    sub["fluid_group"],
    normalize="columns"
).round(3)


fluid_group,High,Low
sofa_renal,,
0.0,0.073,0.101
1.0,0.259,0.284
2.0,0.247,0.342
3.0,0.216,0.181
4.0,0.205,0.092


In [15]:
import numpy as np

def smd(x1, x0):
    return (x1.mean() - x0.mean()) / np.sqrt((x1.var() + x0.var()) / 2)

smd_renal = smd(
    gH["sofa_renal"].dropna(),
    gL["sofa_renal"].dropna()
)

print(f"SMD renal SOFA = {smd_renal:+.2f}")


SMD renal SOFA = +0.29


In [16]:
sub = df_fg[
    (df_fg["sofa"] >= 8) 
].copy()

from scipy.stats import fisher_exact

rows = []

for r in sorted(sub["sofa_renal"].dropna().unique()):
    d = sub[sub["sofa_renal"] == r]

    gH = d[d["fluid_group"] == "High"]
    gL = d[d["fluid_group"] == "Low"]

    if len(gH) < 20 or len(gL) < 20:
        continue  # zu wenig Power

    rate_H = gH["rrt_persistent_last6h"].mean()
    rate_L = gL["rrt_persistent_last6h"].mean()

    a = gH["rrt_persistent_last6h"].sum()
    b = len(gH) - a
    c = gL["rrt_persistent_last6h"].sum()
    d_ = len(gL) - c

    _, p = fisher_exact([[a, b], [c, d_]])

    rows.append({
        "renal_sofa": r,
        "n_High": len(gH),
        "n_Low": len(gL),
        "RRT_High_%": rate_H * 100,
        "RRT_Low_%": rate_L * 100,
        "Delta_pp": (rate_H - rate_L) * 100,
        "p_value": p
    })

pd.DataFrame(rows)


,renal_sofa,n_High,n_Low,RRT_High_%,RRT_Low_%,Delta_pp,p_value
0,0.0,63,63,15.873016,0.000000,15.873016,1.326765e-03
1,1.0,238,189,6.302521,0.000000,6.302521,2.033233e-04
2,2.0,250,228,13.600000,0.000000,13.600000,8.807803e-11
3,3.0,260,129,21.923077,2.325581,19.597496,2.948172e-08
4,4.0,274,85,40.875912,0.000000,40.875912,9.224518e-17


In [17]:
import numpy as np
import pandas as pd

sub = df_fg[
    (df_fg["sofa"] >= 8) &
    (df_fg["sofa"] <= 12)
].copy()

matched_blocks = []

for r in sorted(sub["sofa_renal"].dropna().unique()):
    d = sub[sub["sofa_renal"] == r]

    gH = d[d["fluid_group"] == "High"]
    gL = d[d["fluid_group"] == "Low"]

    n = min(len(gH), len(gL))
    if n < 20:
        continue  # zu wenig Power

    gH_s = gH.sample(n, random_state=42)
    gL_s = gL.sample(n, random_state=42)

    matched_blocks.append(pd.concat([gH_s, gL_s]))

matched_df = pd.concat(matched_blocks)

print("Matched N:", len(matched_df))
print(matched_df["fluid_group"].value_counts())
print(matched_df["sofa_renal"].value_counts().sort_index())


Matched N: 1188
fluid_group
High    594
Low     594
Name: count, dtype: int64
sofa_renal
0.0    116
1.0    348
2.0    390
3.0    222
4.0    112
Name: count, dtype: int64


In [18]:
g = matched_df.groupby("fluid_group")["hospital_mortality"]

mort_table = pd.DataFrame({
    "n": g.size(),
    "deaths": g.sum(),
    "mortality_%": g.mean() * 100
}).round(1)
print(len(matched_df))
mort_table


1188


,n,deaths,mortality_%
fluid_group,,,
High,594,195,32.8
Low,594,157,26.4


In [19]:
import pandas as pd
import numpy as np

matched_blocks = []

group_cols = ["sofa", "sofa_renal"]

for (s, r), d in sub.groupby(group_cols):
    gH = d[d["fluid_group"] == "High"]
    gL = d[d["fluid_group"] == "Low"]

    n = min(len(gH), len(gL))
    if n < 10:   # Mindestgröße, sonst keine Power
        continue

    gH_s = gH.sample(n, random_state=42)
    gL_s = gL.sample(n, random_state=42)

    matched_blocks.append(pd.concat([gH_s, gL_s]))

matched_df = pd.concat(matched_blocks, ignore_index=True)

print("Matched N total:", len(matched_df))
print(matched_df["fluid_group"].value_counts())


Matched N total: 1110
fluid_group
High    555
Low     555
Name: count, dtype: int64


In [20]:
a = gH["hospital_mortality"].sum()
b = len(gH) - a
c = gL["hospital_mortality"].sum()
d = len(gL) - c

rate_H = a / len(gH) * 100
rate_L = c / len(gL) * 100

_, p_mort = fisher_exact([[a, b], [c, d]])

print(
    f"Hospital mortality: "
    f"High={rate_H:.1f}% vs Low={rate_L:.1f}% "
    f"(Δ {rate_H-rate_L:+.1f}pp), p={p_mort:.4g}"
)


Hospital mortality: High=51.7% vs Low=30.0% (Δ +21.7pp), p=0.2898


In [21]:
from scipy.stats import fisher_exact

gH = matched_df[matched_df["fluid_group"] == "High"]
gL = matched_df[matched_df["fluid_group"] == "Low"]

a = gH["rrt_persistent_last6h"].sum()
b = len(gH) - a
c = gL["rrt_persistent_last6h"].sum()
d = len(gL) - c

rate_H = a / len(gH) * 100
rate_L = c / len(gL) * 100

_, p_rrt = fisher_exact([[a, b], [c, d]])

print(
    f"Persistent RRT: "
    f"High={rate_H:.1f}% vs Low={rate_L:.1f}% "
    f"(Δ {rate_H-rate_L:+.1f}pp), p={p_rrt:.4g}"
)


Persistent RRT: High=14.1% vs Low=0.4% (Δ +13.7pp), p=3.36e-22


In [22]:
def median_iqr(x):
    return pd.Series({
        "median": x.mean(),
        "q25": x.quantile(0.25),
        "q75": x.quantile(0.75)
    })

vars_to_check = ["age", "sofa", "sofa_renal"]

summary = (
    matched_df
    .groupby("fluid_group")[vars_to_check]
    .apply(lambda df: df.apply(median_iqr))
)

summary

age       sofa  sofa_renal
fluid_group                                         
High        median  64.783784   9.418018    1.861261
            q25     55.000000   8.000000    1.000000
            q75     77.000000  10.000000    3.000000
Low         median  62.374775   9.418018    1.861261
            q25     52.000000   8.000000    1.000000
            q75     74.000000  10.000000    3.000000

In [23]:
pd.crosstab(
    matched_df["sofa_renal"],
    matched_df["fluid_group"],
    normalize="columns"
).round(3)


fluid_group,High,Low
sofa_renal,,
0.0,0.083,0.083
1.0,0.314,0.314
2.0,0.333,0.333
3.0,0.200,0.200
4.0,0.070,0.070


In [24]:
# Wie viele Zeilen vs. wie viele eindeutige icustay_ids?
matched_df.shape[0], matched_df["icustay_id"].nunique()


(1110, 1110)

In [25]:
groups_per_id = (
    matched_df
    .groupby("icustay_id")["fluid_group"]
    .nunique()
)

# Gibt es icustay_ids mit mehr als einer Gruppe?
groups_per_id.value_counts()


fluid_group
1    1110
Name: count, dtype: int64

In [28]:
from src.utils import add_sapsii_score
matched_df = add_sapsii_score(matched_df)


In [31]:
from scipy.stats import mannwhitneyu

gH = matched_df[matched_df["fluid_group"] == "High"]
gL = matched_df[matched_df["fluid_group"] == "Low"]

# Median & IQR
print(
    f"SAPS II predicted mortality (median, IQR): "
    f"High={gH['sapsii_prob'].median():.3f} "
    f"({gH['sapsii_prob'].quantile(0.25):.3f}-"
    f"{gH['sapsii_prob'].quantile(0.75):.3f}) | "
    f"Low={gL['sapsii_prob'].median():.3f} "
    f"({gL['sapsii_prob'].quantile(0.25):.3f}-"
    f"{gL['sapsii_prob'].quantile(0.75):.3f})"
)

# Test
stat, p = mannwhitneyu(
    gH["sapsii_prob"].dropna(),
    gL["sapsii_prob"].dropna(),
    alternative="two-sided"
)

print(f"Mann–Whitney U p-value = {p:.4g}")


SAPS II predicted mortality (median, IQR): High=0.484 (0.326-0.681) | Low=0.392 (0.266-0.598)
Mann–Whitney U p-value = 2.531e-07
